# Dev 03 - Geographic and Cartesian Coordinates

In [6]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('./darkmode.style')

In [2]:
# Radar coordinate
# Approximate location from Google Maps: 39.7294019,140.0440751
olat, olon = 39.7294019, 140.0440751

In [7]:
def geo2cart(lat, lon, olat, olon):
    # Convention:
    # Longitude = lam = angle from the x-axis
    # Latitude = phi = angle from the xy-plane
    e = 6371e3
    lam = np.deg2rad(lon)
    phi = np.deg2rad(lat)
    p = np.array([
        np.cos(phi) * np.cos(lam),
        np.cos(phi) * np.sin(lam),
        np.sin(phi)
    ])
    # Step 1 - rotate the earth about the z-axis by -(90 + lon)
    az = np.deg2rad(-(90 + olon))
    rz = np.array([[np.cos(az), -np.sin(az), 0], [np.sin(az), np.cos(az), 0], [0, 0, 1]])
    # Step 2 - rotate the earth about the x-axis by -(90 - lat)
    ax = np.deg2rad(-(90 - olat))
    rx = np.array([[1, 0, 0], [0, np.cos(ax), -np.sin(ax)], [0, np.sin(ax), np.cos(ax)]])
    # Matrix multiplication form
    q = e * np.matmul(rx, np.matmul(rz, p))
    return q[0], q[1], q[2] - e

In [13]:
lat, lon = olat + 0.01, olon

x, y, z = geo2cart(lat + 0.01, lon, olat=olat, olon=olon)

print('x = {:.4f}   y = {:.4f}   z = {:.4f}'.format(x, y, z))

x = 0.0000   y = 2223.8985   z = -0.3881


In [14]:
def cart2geo(x, y, z, olat, olon):
    e = 6371e3
    # Step 1 - rotate the earth about the x-axis by (90 - lat)
    ax = np.deg2rad(90 - olat)
    rx = np.array([[1, 0, 0], [0, np.cos(ax), -np.sin(ax)], [0, np.sin(ax), np.cos(ax)]])
    # Step 2 - rotate the earth about the z-axis by (90 + lon)
    az = np.deg2rad(90 + olon)
    rz = np.array([[np.cos(az), -np.sin(az), 0], [np.sin(az), np.cos(az), 0], [0, 0, 1]])
    q = np.array([x, y, z + e])
    p = np.matmul(rz, np.matmul(rx, q))
    x, y, z = p
    h = np.sqrt(x * x + y * y)
    lam = np.arctan2(y, x)
    phi = np.arctan2(z, h)
    lat, lon = np.rad2deg(phi), np.rad2deg(lam)
    return lat, lon

In [22]:
x, y, z = 0, 1112, -0.1
# lat, lon = 39.7294019, 140.0440751
plat, plon = cart2geo(x, y, z, olat=olat, olon=olon)
print('olat = {:.4f}   olon = {:.4f}'.format(olat, olon))
print('plat = {:.4f}   plon = {:.4f}'.format(plat, plon))

olat = 39.7294   olon = 140.0441
plat = 39.7394   plon = 140.0441
